In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import string
import numpy as np
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import json
import ast
from  more_itertools import unique_everseen
from tqdm import tqdm
import re
import time

import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%matplotlib inline
import matplotlib.pyplot as plt

load data

In [3]:
datapath=''
# datapath='../data/modifled/'

In [15]:
%%time
df=pd.read_hdf(datapath+'df.h5', 'table')

CPU times: user 1.03 s, sys: 573 ms, total: 1.6 s
Wall time: 1.6 s


In [16]:
df['From'].value_counts()

shopstyle    955818
farfetch     106705
nap            9627
Name: From, dtype: int64

In [17]:
desc=df['comb_desc']

use spacy to preprocess description

In [7]:
nlp = spacy.load('en_core_web_lg')

In [8]:
# stopwords
SYMBOLS = set((list(string.punctuation) + ['...']))
# BRAND_NAME = sorted(map(lambda x:x,set(df['Brand Name'])))
STOP_WORDS = set(ENGLISH_STOP_WORDS) | STOP_WORDS #| set(BRAND_NAME)
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [9]:
def pipe_tokenizer(df):
    def token_filter(token):
        return not (token.is_punct | token.is_space | token.is_oov | token.is_digit
                     | len(token.text) <= 2 )
    tokens = []
    for doc in nlp.pipe(df.values, batch_size=600, disable=['parser','ner']):
        token = [token.lemma_.strip() for token in doc if (token_filter(token) and not token.is_stop)]
        token = ' '.join(token)
        tokens.append(token)
    return tokens

In [20]:
start_time = time.time()
tokens = pipe_tokenizer(desc)
print("--- %.2f seconds ---" % ((time.time() - start_time)/60))

--- 39.72 seconds ---


In [21]:
df['comb_desc'] = tokens

In [22]:
df.to_hdf('df.h5','table')